In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
import data_generator
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from path import col_seq_can, col_seq_can_all
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [5]:
%config Completer.use_jedi = False

### Load Pmpm and IDs

In [6]:
# import boto3
# s3 = boto3.client('s3')
# bucket = 'cdphp-s3-us-e-p-pond'
# path_to_save = '../../../data/PMPM.p'
# s3_path = 'rpi/nneehal/DATA/pmpm_202108_processed_ready.p'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [7]:
# path_to_save = '../../../data/ids.csv'
# s3_path = 'rpi/blind_matching_study/input_data/RPI Test Cohort.csv'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [8]:
%%time
ids_csv = '../../../data/ids.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.75 s, sys: 10.9 s, total: 19.6 s
Wall time: 19.6 s


In [9]:
treated_ids.shape

### Drop Unecessary data points

In [10]:
pmpm_backup = pmpm.copy()

In [11]:
pmpm_backup.MYR.min(), pmpm_backup.MYR.max()

(201701, 202108)

In [12]:
#pmpm_backup2 = pmpm_backup[pmpm_backup.MYR >= 202003].reset_index(drop = True)

In [13]:
pmpm_backup.shape

In [14]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [15]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [16]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [17]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [18]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [19]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [20]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 486 ms, sys: 309 ms, total: 795 ms
Wall time: 794 ms


In [21]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

In [22]:
treated.drop(['MEDICAID_LOB'], axis=1, inplace=True)
controls.drop(['MEDICAID_LOB'], axis=1, inplace=True)

# Take average of treated and controls

### Generate Treated

In [23]:
treated2 = data_process.get_average_record(df=treated, pmpm=pmpm_backup.copy())

PERS_ID
First date
One year before
history from original pmpm
record size calculated
record filtered
returning from f1...
f1 done


100%|██████████| 97/97 [00:00<00:00, 37260.51it/s]

f2 done
stacking done
returning..


In [24]:
treated2.ALLOW_AMT = treated2.ALLOW_AMT.apply(data_process.log_)
treated2.ALLOW_IP = treated2.ALLOW_IP.apply(data_process.log_)
treated2.ALLOW_ER = treated2.ALLOW_ER.apply(data_process.log_)

In [25]:
#treated2.to_pickle("jeremy_avg_treated.p")

### Generate Controls 

In [26]:
controls.copy()

PERS_ID     MYR  MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0             -3  202103         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1             -3  202104         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2             -3  202105         1              121          0       0.0      0.00      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3             -3  202106         1           

In [27]:
controls2 = data_process.get_average_controls_jeremy(controls.copy(), pmpm_backup.copy())

[-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
(158288, 92)
(158288, 77)
[-13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2]
(162237, 92)
(320525, 77)
[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3]
(165840, 92)
(486365, 77)
[-15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4]
(168384, 92)
(654749, 77)
[-16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]
(169947, 92)
(824696, 77)
[-17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6]
(169841, 92)
(994537, 77)


In [28]:
controls2.ALLOW_AMT = controls2.ALLOW_AMT.apply(data_process.log_)
controls2.ALLOW_IP = controls2.ALLOW_IP.apply(data_process.log_)
controls2.ALLOW_ER = controls2.ALLOW_ER.apply(data_process.log_)

In [29]:
#controls2.to_pickle("jeremy_avg_controls.p")

### Save in S3 for reuse

In [30]:
# !aws s3 cp jeremy_avg_treated.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_treated.p --acl bucket-owner-full-control
# !aws s3 cp jeremy_avg_controls.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_controls.p --acl bucket-owner-full-control

In [31]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/get_matches1_8.ipynb s3_get_matches1_8.ipynb --acl bucket-owner-full-control

In [32]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/mapping.xlsx mapping.xlsx --acl bucket-owner-full-control

## Match begin

In [33]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [34]:
def get_data(data_struct, cont):
    cont = cont.copy()
    cont.PERS_ID = cont.PERS_ID.astype(str)
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [35]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [36]:
cols_to_keep = ["PERS_ID", "MYR"]

## Load Model

In [37]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked_logfixed", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [38]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [39]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 10.9 s, sys: 2.64 s, total: 13.5 s
Wall time: 7.92 s


## Scenario 1 Rigorous

In [40]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st1, matched_data1 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st1, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:28,  3.46it/s]


CPU times: user 1min 41s, sys: 1min 18s, total: 2min 59s
Wall time: 30.9 s


In [41]:
dat1.shape

## Scenario 2 Non Rigorous

In [42]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st2, matched_data2 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st2, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:28,  3.38it/s]


CPU times: user 1min 43s, sys: 1min 21s, total: 3min 4s
Wall time: 31.7 s


In [43]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [44]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st3, matched_data3 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st3, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:50,  1.93it/s]


CPU times: user 2min, sys: 1min 27s, total: 3min 27s
Wall time: 53.4 s


In [45]:
dat3.MYR.min(), dat3.MYR.max()

In [46]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [47]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st4, matched_data4 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st4, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:51,  1.92it/s]


CPU times: user 2min 1s, sys: 1min 26s, total: 3min 28s
Wall time: 53.8 s


In [48]:
dat4.shape

## Scenario 5 Medicaid Rigorous 

In [49]:
#original single controls who had LOB=0 in single month
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

#extract those ids who have aggregate features in controls2 and LOB=0 in original controls
controlsM_with_agg_features_id = controlsM[controlsM.PERS_ID.isin(controls2.PERS_ID)].PERS_ID.unique()

#extract those aggregate features
controlsM_with_agg_features = controls2[controls2.PERS_ID.astype(int).isin(controlsM_with_agg_features_id)].reset_index(drop=True)

#transform using neural network for those aggregate features
cntrlsM = data_process.transform_features_with_NN(model, controlsM_with_agg_features[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

#final original controls
controlsM = controls[controls.PERS_ID.isin(controlsM_with_agg_features_id)]

In [50]:
cntrlsM.PERS_ID = cntrlsM.PERS_ID.astype(str)

In [51]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st5, matched_data5 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st5, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:11,  8.61it/s]


CPU times: user 1min 21s, sys: 1min 13s, total: 2min 34s
Wall time: 13.7 s


In [52]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [53]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st6, matched_data6 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st6, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:11,  8.53it/s]


CPU times: user 1min 23s, sys: 1min 15s, total: 2min 38s
Wall time: 13.9 s


In [54]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [55]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st7, matched_data7 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st7, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:15,  6.46it/s]


CPU times: user 1min 30s, sys: 1min 17s, total: 2min 48s
Wall time: 17.5 s


In [56]:
dat7.MYR.min(), dat7.MYR.max()

In [57]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [58]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st8, matched_data8 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st8, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:39305 before they were:39305


98it [00:15,  6.45it/s]


CPU times: user 1min 30s, sys: 1min 18s, total: 2min 48s
Wall time: 17.5 s


In [59]:
dat8.shape

# Save Stats

In [60]:
from utils import statistics_first_month
import scipy.stats as sts

In [61]:
columns_to_match = treated.columns.tolist()

In [62]:
treated2.head()

PERS_ID     MYR AGE_AT_MIDMONTH LOB MBR_GNDR  ALLOW_AMT   ALLOW_ER  ALLOW_IP CC_ADHD CC_ALZHEIMER CC_ALZHEIMER_PLUS CC_AMI CC_ANEMIA CC_ANXIETY_DISORDER CC_ARTHRITIS CC_ASTHMA CC_ATRIAL_FIB CC_AUTISM CC_BIPOLAR    CC_BPH CC_BREAST_CANCER CC_CATARACT CC_CEREBRAL_PALSY CC_CHRONIC_KIDNEY CC_COLORECTAL_CANCER CC_COPD CC_CYSTIC_FIBROSIS CC_DEPRESSION CC_DEPRESSIVE_DISORDERS CC_DEVELOP_DELAYS CC_DIABETES CC_DISAB_DX_CNT CC_DX_CNT CC_ENDOMETRIAL_CANCER CC_EPILEPSY CC_FIBROMYALGIA CC_GLAUCOMA CC_HEARING_IMPAIR CC_HEART_FAILURE CC_HEPATITIS_A CC_HEPATITIS_B_ACUTE CC_HEPATITIS_B_CHRONIC CC_HEPATITIS_C_ACUTE CC_HEPATITIS_C_CHRONIC CC_HEPATITIS_C_UNSPECIFIED CC_HEPATITIS_D CC_HEPATITIS_E CC_HEPATITIS_GEN CC_HIP_FRACTURE CC_HYPERLIPIDEMIA CC_HYPERTENSION CC_HYPOTHYROID CC_INT_DISAB CC_ISCHEMIC CC_LEARN_DISAB CC_LEUKEMIAS  CC_LIVER CC_LUNG_CANCER CC_MIGRAINE CC_MOBILITY_IMPAIR CC_MULTIPLE_SCLEROSIS CC_MUSCULAR_DYSTROPHY CC_OBESITY CC_OSTEOPOROSIS CC_PERIPHERAL CC_PERSONALITY CC_PRESSURE  \
0  2778075  202108         62.7692   0        0   7.905130 -16.118096  6.133500       0            0                 0      0         0            0.538462            0         0             0         0          0         0                0           0                 0          0.307692                    0       0                  0      0.769231                       0                 0           1               0   4.38462                     0           0        0.307692           0                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0          0.307692        0.307692              0            0           0              0            0  0.846154              0           0                  0                     0                     0          0               0             0              0           0   
1  2824337  202108         59.3077   0        1   7.949499 -16.118096  6.234147       0            0                 0      0         0                   1            0         0             0         0   0.307692  0.307692                0    0.923077                 0                 1                    0       0                  0             1                       0                 0           1         1.15385   9.46154                     0           0        0.923077           0                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0                 1               1              0            0           0              0            0         0              0           0           0.307692                     0                     0          0               0             0              0           0   
2  2826151  202108         59.1538   0        0   8.373856   5.796006  6.986582       0            0                 1      0         0                   1            1         0             0         0          1         0                0           0                 0          0.692308                    0       1                  0             1                       1                 0           0               0   15.9231                     0           0               1    0.538462                 0                0              0                    0                      0                    0                      0                          0              0              0                0               0                 0               0              0            0           0              0            0         0              0           1                  0                     0                     0          1               0             0              1           0   
3  2851787  202108         

In [63]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



CPU times: user 1min 26s, sys: 2.19 s, total: 1min 28s
Wall time: 1min 27s


In [64]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [65]:
pd.options.display.float_format = '{:.2f}'.format

In [66]:
alli[1:]

Treated         Unmatched Controls              Rig         NonRig            Rig6         NonRig6         Unmatched Med_Controls           Rig-Med         NonRig-Med         Rig-Med6         NonRig-Med6        
                              mean     std               mean       std   mean     std   mean     std    mean     std    mean     std                   mean       std    mean     std       mean     std     mean     std        mean     std
MBR_GNDR                      0.34    0.48               0.44      0.50   0.35    0.48   0.35    0.48    0.33    0.47    0.36    0.48                   0.38      0.48    0.33    0.47       0.31    0.46     0.32    0.47        0.32    0.47
AGE_AT_MIDMONTH              37.87   11.87              48.09     20.25  37.88   11.66  37.21   11.33   38.57   11.74   38.18   11.71                  35.80     16.51   37.55   11.37      37.02   11.09    38.18   11.86       37.89   11.45
ALLOW_AMT                   709.13 1425.41            1030.41   5321.78 679.21 3079.22 388.98 1815.73 1836.20 5348.81 2114.70 8341.12                 838.73   2731.39  426.27 1907.30     288.22 1227.06  1740.78 4384.91     1406.13 4426.47
ALLOW_ER                     22.19  110.86              20.08    192.30  28.15  152.45  18.44  123.51   54.35  244.20   55.37  252.75                  19.90    127.93   18.88   99.78      14.28   91.28    57.52  186.39       48.23  202.72
ALLOW_IP                    279.28 1205.83             249.13   4264.12 211.51 2118.31  79.18 1196.53  787.18 4307.10  930.67 6885.28                 155.67   1657.18  106.55 1024.96      47.25  696.82   697.30 3306.70      482.61 3492.22
CC_ADHD                       0.23    0.42               0.04      0.20   0.07    0.26   0.06    0.24    0.07    0.26    0.09    0.28                   0.09      0.29    0.08    0.27       0.08    0.27     0.11    0.31        0.10    0.30
CC_ALZHEIMER                  0.01    0.09               0.01      0.07   0.00    0.00   0.00    0.00    0.00    0.00    0.00    0.02                   0.00      0.03    0.00    0.00       0.00    0.05     0.00    0.05        0.00    0.04
CC_ALZHEIMER_PLUS             0.04    0.19               0.02      0.13   0.01    0.09   0.00    0.07    0.01    0.12    0.01    0.12                   0.01      0.09    0.01    0.09       0.01    0.08     0.03    0.17        0.02    0.13
CC_AMI                        0.02    0.13               0.00      0.06   0.02    0.13   0.01    0.08    0.01    0.08    0.01    0.12                   0.00      0.05    0.01    0.08       0.01    0.08     0.02    0.13        0.01    0.10
CC_ANEMIA                     0.16    0.37               0.07      0.26   0.18    0.38   0.12    0.33    0.19    0.39    0.16    0.37                   0.08      0.28    0.15    0.36       0.12    0.32     0.20    0.40        0.16    0.36
CC_ANXIETY_DISORDER           0.90    0.30               0.22      0.42   0.41    0.49   0.37    0.48    0.52    0.50    0.46    0.50                   0.34      0.47    0.43    0.50       0.41    0.49     0.55    0.50        0.49    0.50
CC_ARTHRITIS                  0.14    0.35               0.14      0.34   0.11    0.31   0.10    0.30    0.17    0.38    0.13    0.34                   0.10      0.30    0.12    0.32       0.11    0.31     0.17    0.38        0.15    0.36
CC_ASTHMA                     0.21    0.41               0.06      0.23   0.16    0.36   0.11    0.31    0.19    0.39    0.15    0.36                   0.10      0.30    0.16    0.37       0.12    0.33     0.21    0.41        0.18    0.38
CC_ATRIAL_FIB                 0.04    0.19               0.03      0.17   0.01    0.10   0.01    0.10    0.02    0.14    0.02    0.14                   0.01      0.10    0.01    0.09       0.00    0.07     0.02    0.14        0.01    0.12
CC_AUTISM                     0.04    0.19               0.01      0.08   0.00    0.05   0.01    0.07    0.01    0.11    0.01    0.10                   0.02      0.13    0.00    0.06       0.01    0.10  

In [67]:
Path('./results/stats_mean_std').mkdir(exist_ok = True)
alli[1:].to_csv("./results/stats_mean_std/seq_9_16_mean_std.csv", header=True, index=True)

## Latent Stats

In [68]:
dat_lists = [matched_data1, matched_data2, matched_data3, matched_data4, matched_data5, matched_data6, matched_data7, matched_data8]
stat = []
for data in dat_lists:
#     dat_agg = data_process.get_average_controls_jeremy(data, pmpm_backup.copy())
#     dat_lat = data_process.transform_features_with_NN(model, dat_agg[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    stat_a, N, _ = statistics_first_month(data, latent_cols, return_array = True)
    stat_a = stat_a.T
    stat.append(stat_a)

In [69]:
tr_stats, N, _ =  statistics_first_month(tr2, latent_cols, return_array = True)
tr_stats = tr_stats.T

In [70]:
alli_latent = pd.concat((tr_stats[['mean','std']], stat[0][['mean','std']], stat[1][['mean','std']], stat[2][['mean','std']], stat[3][['mean','std']], 
                  stat[4][['mean','std']], stat[5][['mean','std']], stat[6][['mean','std']], stat[7][['mean','std']]),
                 axis = 1, keys = ['Treated', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7','seq8'])

In [71]:
alli_latent

Treated       seq1       seq2       seq3       seq4       seq5       seq6       seq7       seq8     
       mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std  mean  std
x1     1.09 1.84  1.08 1.84  1.09 1.83  0.92 1.88  0.92 1.87  1.09 1.83  1.07 1.84  0.98 1.86  0.97 1.85
x2     0.41 2.67  0.39 2.66  0.40 2.65  0.17 2.74  0.17 2.73  0.41 2.65  0.38 2.66  0.25 2.70  0.24 2.70
x3    -1.16 2.16 -1.15 2.15 -1.16 2.15 -0.97 2.20 -0.96 2.20 -1.16 2.14 -1.14 2.15 -1.03 2.18 -1.02 2.17
x4    -0.40 0.39 -0.40 0.39 -0.40 0.38 -0.36 0.39 -0.36 0.39 -0.40 0.38 -0.39 0.39 -0.37 0.39 -0.37 0.39
x5     0.54 3.76  0.56 3.76  0.55 3.74  0.86 3.88  0.87 3.88  0.53 3.74  0.58 3.75  0.74 3.81  0.76 3.82
x6     1.11 1.10  1.10 1.09  1.10 1.09  1.00 1.11  1.00 1.11  1.10 1.09  1.09 1.10  1.03 1.11  1.02 1.10
x7     0.34 0.22  0.33 0.22  0.34 0.22  0.31 0.23  0.31 0.23  0.34 0.22  0.33 0.22  0.32 0.23  0.32 0.23
x8     0.85 0.91  0.84 0.91  0.85 0.90  0.76 0.92  0.76 0.91  0.85 0.90  0.84 0.91  0.79 0.92  0.78 0.91
x9     1.09 0.83  1.09 0.83  1.09 0.83  1.01 0.84  1.01 0.84  1.09 0.83  1.08 0.83  1.04 0.84  1.03 0.84
x10   -0.74 1.76 -0.73 1.76 -0.74 1.75 -0.58 1.79 -0.58 1.79 -0.74 1.75 -0.72 1.76 -0.63 1.78 -0.62 1.77
x11   -1.05 0.97 -1.05 0.96 -1.05 0.96 -0.96 0.98 -0.96 0.97 -1.05 0.96 -1.04 0.97 -0.99 0.98 -0.98 0.97
x12    0.84 1.90  0.83 1.89  0.83 1.89  0.67 1.94  0.66 1.93  0.84 1.89  0.81 1.89  0.72 1.92  0.71 1.91
x13    1.22 0.96  1.22 0.96  1.22 0.96  1.13 0.97  1.13 0.97  1.22 0.96  1.21 0.96  1.16 0.97  1.15 0.96
x14   -1.16 1.03 -1.15 1.03 -1.15 1.03 -1.06 1.04 -1.05 1.04 -1.15 1.03 -1.14 1.03 -1.09 1.04 -1.08 1.03
x15   -1.73 1.22 -1.72 1.22 -1.73 1.22 -1.61 1.24 -1.61 1.23 -1.73 1.22 -1.71 1.22 -1.65 1.24 -1.64 1.23
x16   -0.44 2.28 -0.43 2.28 -0.44 2.27 -0.24 2.34 -0.23 2.33 -0.44 2.27 -0.41 2.28 -0.30 2.31 -0.29 2.30

In [72]:
Path('./results/latent_stats_mean_std').mkdir(exist_ok = True)
alli_latent.to_csv("./results/latent_stats_mean_std/seq_9_16_latent_mean_std.csv", index=True, header=True)

## Save latent features

In [75]:
dat_lists = [matched_data1, matched_data2, matched_data3, matched_data4, matched_data5, matched_data6, matched_data7, matched_data8]
file_id_start = 9
Path('./results/latent_features').mkdir(exist_ok = True)
for data in dat_lists:
    data.drop(['MYR'], axis=1).to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [76]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq9", 1:"seq10", 2:"seq11", 3:"seq12", 4:"seq13", 5:"seq14", 6:"seq15", 7:"seq16"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [77]:
ids

seq9    seq10    seq11    seq12    seq13    seq14    seq15    seq16
0     2781696  2781696  4288583  4288583  2784384  2784384  2781394  2781394
1     2788482  2788482  3337917  3337917  2821499  2821499  2780869  2780869
2     5023990  5023990  4000358  4000358  5052926  5052926  3647106  3647106
3     2787074  2787074  2781394  2781394  2816482  2816482  4009530  4009530
4     2784384  2784384  2780869  2780869  5028760  5028760  3693608  3693608
...       ...      ...      ...      ...      ...      ...      ...      ...
1955       -1  4087685       -1  3993335       -1  3536634       -1  4333118
1956       -1  4064686       -1  3687167       -1  2895319       -1  4150128
1957       -1  2919391       -1  2888656       -1  2924297       -1  2899669
1958       -1  2927583       -1  5018340       -1  2912524       -1  4202303
1959       -1  2919687       -1  2856757       -1  2918866       -1  2895078

[1960 rows x 8 columns]

In [78]:
Path('./results/ids').mkdir(exist_ok = True)
ids.to_csv("./results/ids/ids_seq_9_16.csv", index=True, header=True)

## Ratios with/without Std

In [79]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]

In [80]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [81]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [82]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [83]:
std = 1

In [84]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [85]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [86]:
d_metric_after_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.00       0.02      0.05      0.04     0.44          0.13               0.19    0.01       0.05                 1.21          0.10       0.13           0.16       0.25        0.69    0.08              0.13         0.16               0.06               0.30                  0.03     0.32                0.06           0.97                     0.39               0.03         0.11             0.27       0.89                   0.06         0.18             0.38         0.03               0.19              0.21             nan                  0.06                     nan                   nan                    0.05                        0.12             nan             nan              0.11              nan               0.01             0.10            0.03          0.11         0.04            0.07          0.11      0.18            0.09         0.00                0.12                   0.09                    nan        0.12             0.09           0.12   
1      0.02             0.06       0.20      0.03      0.17     0.47          0.13               0.22    0.12       0.10                 1.32          0.14       0.26           0.16       0.21        0.77    0.03              0.05         0.16               0.05               0.44                  0.08     0.41                0.10           1.14                     0.41               0.08         0.21             0.31       1.19                   0.09         0.21             0.47         0.00               0.20              0.32             nan                  0.03                     nan                   nan                    0.12                        0.15             nan             nan              0.17              nan               0.12             0.23            0.08          0.12         0.19            0.13          0.10      0.32            0.05         0.02                0.12                   0.11                   0.03        0.02             0.08           0.07   
2      0.01             0.03       0.28      0.16      0.15     0.45          0.13               0.14    0.11       0.08                 0.93          0.06       0.03           0.08       0.17        0.67    0.20              0.03         0.18               0.09               0.28                  0.13     0.14                0.09           0.79                     0.30               0.06         0.02             0.10       0.64                   0.09         0.11             0.26         0.04               0.25              0.16             nan                   nan                    0.06                   nan                    0.07                        0.04             nan             nan              0.05              nan               0.05             0.01            0.07          0.09         0.03            0.13          0.11      0.16             nan         0.10                0.00                   0.13                    nan        0.24             0.03           0.12   
3  

In [87]:
d_metric_before_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.19             0.70       0.04      0.05      0.04     0.56          0.04               0.11    0.14       0.27                 1.82          0.01       0.45           0.01       0.21        1.01    0.16              0.08         0.39               0.04               0.42                  0.06     0.34                0.07           1.47                     0.45               0.10         0.12             0.32       1.23                   0.10         0.27             0.57         0.15               0.28              0.23            0.01                  0.03                    0.04                  0.01                    0.15                        0.21            0.01            0.00              0.22             0.06               0.20             0.02            0.01          0.15         0.07            0.15          0.12      0.41            0.09         0.18                0.15                   0.10                   0.03        0.11             0.10           0.21   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.63         0.16                0.15     1.25              0.44                    0.53            0.16             0.05       0.04        1.23                0.09              0.05 2.58       0.04    0.29    0.22   0.35  1.11  1.13     1.31     1.03     0.88    1.29     0.59     0.56    0.77

In [88]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [89]:
ratio_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.00       0.43      0.91      0.95     0.79          3.68               1.80    0.09       0.17                 0.66          7.29       0.29          12.46       1.21        0.68    0.51              1.71         0.41               1.53               0.70                  0.54     0.95                0.88           0.66                     0.85               0.33         0.91             0.82       0.72                   0.63         0.66             0.67         0.19               0.70              0.91             nan                  2.39                     nan                   nan                    0.32                        0.59             nan             nan              0.50              nan               0.07             4.33            2.75          0.74         0.60            0.45          0.93      0.45            1.01         0.02                0.85                   0.91                    nan        1.08             0.92           0.58   
1      0.09             0.08       5.40      0.62      3.86     0.85          3.68               2.12    0.81       0.39                 0.73         10.96       0.59          13.14       1.04        0.76    0.20              0.59         0.42               1.08               1.04                  1.25     1.21                1.33           0.77                     0.91               0.76         1.79             0.95       0.96                   0.97         0.80             0.82         0.02               0.71              1.38             nan                  1.19                     nan                   nan                    0.77                        0.71             nan             nan              0.77              nan               0.62             9.50            8.53          0.78         2.83            0.92          0.80      0.77            0.50         0.09                0.85                   1.12                   1.23        0.18             0.75           0.35   
2      0.03             0.04       7.51      3.12      3.37     0.80          3.68               1.31    0.77       0.29                 0.51          4.89       0.07           6.47       0.84        0.67    1.25              0.42         0.47               2.17               0.66                  2.19     0.41                1.25           0.54                     0.66               0.61         0.21             0.31       0.52                   0.97         0.41             0.46         0.29               0.91              0.69             nan                   nan                    1.74                   nan                    0.44                        0.21             nan             nan              0.25              nan               0.27             0.50            6.68          0.59         0.48            0.92          0.93      0.39             nan         0.58                0.02                   1.29                    nan        2.21             0.32           0.58   
3  

In [90]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [91]:
mask_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.00       0.43      0.91      0.95     0.79           nan                nan    0.09       0.17                 0.66           nan       0.29            nan        nan        0.68    0.51               nan         0.41                nan               0.70                  0.54     0.95                0.88           0.66                     0.85               0.33         0.91             0.82       0.72                   0.63         0.66             0.67         0.19               0.70              0.91             nan                   nan                     nan                   nan                    0.32                        0.59             nan             nan              0.50              nan               0.07              nan             nan          0.74         0.60            0.45          0.93      0.45             nan         0.02                0.85                   0.91                    nan         nan             0.92           0.58   
1      0.09             0.08        nan      0.62       nan     0.85           nan                nan    0.81       0.39                 0.73           nan       0.59            nan        nan        0.76    0.20              0.59         0.42                nan                nan                   nan      nan                 nan           0.77                     0.91               0.76          nan             0.95       0.96                   0.97         0.80             0.82         0.02               0.71               nan             nan                   nan                     nan                   nan                    0.77                        0.71             nan             nan              0.77              nan               0.62              nan             nan          0.78          nan            0.92          0.80      0.77            0.50         0.09                0.85                    nan                    nan        0.18             0.75           0.35   
2      0.03             0.04        nan       nan       nan     0.80           nan                nan    0.77       0.29                 0.51           nan       0.07            nan       0.84        0.67     nan              0.42         0.47                nan               0.66                   nan     0.41                 nan           0.54                     0.66               0.61         0.21             0.31       0.52                   0.97         0.41             0.46         0.29               0.91              0.69             nan                   nan                     nan                   nan                    0.44                        0.21             nan             nan              0.25              nan               0.27             0.50             nan          0.59         0.48            0.92          0.93      0.39             nan         0.58                0.02                    nan                    nan         nan             0.32           0.58   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [92]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [93]:
mask_1_4.columns[~mask_1_4.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_AMI', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_CATARACT', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_DEVELOP_DELAYS', 'CC_DISAB_DX_CNT', 'CC_DX_CNT', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_C_UNSPECIFIED', 'CC_HEPATITIS_GEN', 'CC_HYPERLIPIDEMIA', 'CC_INT_DISAB', 'CC_LIVER', 'CC_MIGRAINE', 'CC_MOBILITY_IMPAIR', 'CC_OSTEOPOROSIS', 'CC_PERIPHERAL', 'CC_PERSONALITY', 'CC_PROSTATE_CANCER', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_SPINAL_CORD', 'CC_TOBACCO', 'CC_TRAUMATIC_BRAIN', 'LOB', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [94]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [95]:
mask_1_4.columns[mask_1_4.isnull().all()]

Index(['CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_ARTHRITIS', 'CC_ATRIAL_FIB', 'CC_CEREBRAL_PALSY', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_B_CHRONIC', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_HYPOTHYROID', 'CC_MUSCULAR_DYSTROPHY', 'CC_PRESSURE', 'CC_SPINA_BIFIDA', 'CC_VISUAL_IMPAIR', 'PREGNANCY'], dtype='object')

### 1.2 Only Medicaid Controls

In [96]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

In [97]:
d_metric_before_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.07             0.11       0.00      0.04      0.10     0.36          0.11               0.18    0.15       0.23                 1.37          0.11       0.29           0.17       0.12        0.77    0.00              0.04         0.21               0.05               0.47                  0.10     0.31                0.08           1.10                     0.39               0.04         0.16             0.24       1.16                   0.12         0.19             0.48         0.01               0.20              0.31            0.02                  0.04                    0.05                  0.02                    0.08                        0.16            0.01            0.01              0.13             0.04               0.07             0.21            0.08          0.07         0.14            0.09          0.09      0.39            0.06         0.08                0.14                   0.10                   0.03        0.01             0.04           0.09   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.54         0.15                0.06     1.02              0.34                    0.40            0.16             0.07       0.07        0.81                0.07              0.05 0.12       0.06    0.26    0.10   0.24  1.01  0.72     0.95     0.96     0.66    1.06     0.55     0.41    0.61

In [98]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [99]:
d_metric_after_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.02       0.17      0.03      0.16     0.43          0.13               0.19    0.11       0.02                 1.15          0.08       0.11           0.19       0.23        0.55    0.08              0.13         0.16                nan               0.35                  0.09     0.25                0.09           0.85                     0.39               0.03         0.08             0.22       0.84                   0.13         0.10             0.33         0.02               0.18              0.23             nan                   nan                     nan                   nan                    0.01                        0.09             nan             nan              0.08              nan               0.07             0.13            0.09          0.09         0.11            0.07          0.06      0.29            0.09         0.03                0.11                   0.09                   0.06        0.06             0.13           0.13   
1      0.06             0.08       0.32      0.08      0.24     0.42          0.09               0.21    0.12       0.12                 1.21          0.12       0.23           0.22       0.17        0.68    0.01              0.07         0.16               0.05               0.45                  0.11     0.35                0.07           0.99                     0.40               0.06         0.19             0.25       1.10                   0.13         0.18             0.43         0.05               0.19              0.32             nan                  0.03                    0.06                   nan                    0.11                        0.14             nan             nan              0.15              nan               0.15             0.27            0.11          0.05         0.19            0.13          0.10      0.37            0.05         0.02                0.12                   0.12                   0.03        0.01             0.09           0.08   
2      0.04             0.02       0.31      0.24      0.15     0.32          0.09               0.01    0.01       0.07                 0.84          0.08       0.02           0.08       0.14        0.54    0.13              0.06         0.17                nan               0.30                  0.13     0.06                0.09           0.68                     0.31               0.01         0.01             0.02       0.49                   0.09         0.02             0.18         0.12               0.28              0.19             nan                  0.06                    0.06                   nan                    0.05                        0.02             nan             nan              0.03             0.06               0.10             0.04            0.04          0.07         0.07            0.05          0.09      0.23            0.06         0.07                0.04                   0.09                    nan        0.22             0.01           0.21   
3  

In [100]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

In [101]:
ratio_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.38             0.21     571.81      0.70      1.59     1.18          1.18               1.05    0.75       0.08                 0.84          0.73       0.39           1.10       1.98        0.71   19.25              3.75         0.74                nan               0.75                  0.95     0.80                1.14           0.77                     0.98               0.75         0.53             0.90       0.72                   1.10         0.52             0.69         1.60               0.93              0.74             nan                   nan                     nan                   nan                    0.16                        0.57             nan             nan              0.60              nan               1.02             0.61            1.14          1.16         0.81            0.77          0.71      0.73            1.41         0.42                0.78                   0.95                   2.03        4.89             3.32           1.42   
1      0.98             0.72    1071.38      2.07      2.41     1.17          0.83               1.16    0.79       0.51                 0.88          1.07       0.79           1.29       1.45        0.88    1.70              1.92         0.74               0.84               0.97                  1.15     1.13                0.90           0.90                     1.01               1.38         1.18             1.02       0.95                   1.10         0.91             0.89         5.06               0.94              1.02             nan                  0.80                    1.17                   nan                    1.36                        0.92             nan             nan              1.10              nan               2.27             1.28            1.48          0.69         1.36            1.49          1.07      0.95            0.70         0.21                0.85                   1.21                   1.01        1.08             2.32           0.90   
2      0.58             0.18    1032.24      6.39      1.53     0.89          0.83               0.04    0.09       0.29                 0.61          0.70       0.06           0.46       1.20        0.70   31.74              1.70         0.80                nan               0.65                  1.37     0.18                1.14           0.61                     0.78               0.28         0.06             0.09       0.42                   0.77         0.13             0.37        12.42               1.45              0.62             nan                  1.60                    1.17                   nan                    0.59                        0.10             nan             nan              0.23             1.53               1.53             0.21            0.47          0.88         0.50            0.55          1.01      0.58            1.00         0.97                0.32                   0.95                    nan       17.76             0.23           2.29   
3  

In [102]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [103]:
mask_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.38             0.21        nan      0.70       nan      nan           nan                nan    0.75       0.08                 0.84          0.73       0.39            nan        nan        0.71     nan               nan         0.74                nan               0.75                  0.95     0.80                 nan           0.77                     0.98               0.75         0.53             0.90       0.72                    nan         0.52             0.69          nan               0.93              0.74             nan                   nan                     nan                   nan                    0.16                        0.57             nan             nan              0.60              nan                nan             0.61             nan           nan         0.81            0.77          0.71      0.73             nan         0.42                0.78                   0.95                    nan         nan              nan            nan   
1      0.98             0.72        nan       nan       nan      nan          0.83                nan    0.79       0.51                 0.88           nan       0.79            nan        nan        0.88     nan               nan         0.74               0.84               0.97                   nan      nan                0.90           0.90                      nan                nan          nan              nan       0.95                    nan         0.91             0.89          nan               0.94               nan             nan                  0.80                     nan                   nan                     nan                        0.92             nan             nan               nan              nan                nan              nan             nan          0.69          nan             nan           nan      0.95            0.70         0.21                0.85                    nan                    nan         nan              nan           0.90   
2      0.58             0.18        nan       nan       nan     0.89          0.83               0.04    0.09       0.29                 0.61          0.70       0.06           0.46        nan        0.70     nan               nan         0.80                nan               0.65                   nan     0.18                 nan           0.61                     0.78               0.28         0.06             0.09       0.42                   0.77         0.13             0.37          nan                nan              0.62             nan                   nan                     nan                   nan                    0.59                        0.10             nan             nan              0.23              nan                nan             0.21            0.47          0.88         0.50            0.55           nan      0.58            1.00         0.97                0.32                   0.95                    nan         nan             0.23            nan   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [104]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [105]:
mask_5_8.columns[~mask_5_8.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_AMI', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_CATARACT', 'CC_CHRONIC_KIDNEY', 'CC_DEPRESSION', 'CC_DX_CNT', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_HEPATITIS_C_UNSPECIFIED', 'CC_LIVER', 'CC_MIGRAINE', 'CC_MOBILITY_IMPAIR', 'CC_PERSONALITY', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_SPINAL_CORD', 'CC_TOBACCO', 'LOB', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [106]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [107]:
mask_5_8.columns[mask_5_8.isnull().all()]

Index(['ALLOW_AMT', 'CC_AUTISM', 'CC_BPH', 'CC_BREAST_CANCER', 'CC_GLAUCOMA', 'CC_HEPATITIS_A', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_MUSCULAR_DYSTROPHY', 'CC_OBESITY', 'CC_PRESSURE', 'CC_SPINA_BIFIDA', 'CC_VISUAL_IMPAIR', 'PREGNANCY'], dtype='object')

### 1.3 Merge and Save ratio

In [108]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.00       0.43      0.91      0.95     0.79          3.68               1.80    0.09       0.17                 0.66          7.29       0.29          12.46       1.21        0.68    0.51              1.71         0.41               1.53               0.70                  0.54     0.95                0.88           0.66                     0.85               0.33         0.91             0.82       0.72                   0.63         0.66             0.67         0.19               0.70              0.91             nan                  2.39                     nan                   nan                    0.32                        0.59             nan             nan              0.50              nan               0.07             4.33            2.75          0.74         0.60            0.45          0.93      0.45            1.01         0.02                0.85                   0.91                    nan        1.08             0.92           0.58   
1      0.09             0.08       5.40      0.62      3.86     0.85          3.68               2.12    0.81       0.39                 0.73         10.96       0.59          13.14       1.04        0.76    0.20              0.59         0.42               1.08               1.04                  1.25     1.21                1.33           0.77                     0.91               0.76         1.79             0.95       0.96                   0.97         0.80             0.82         0.02               0.71              1.38             nan                  1.19                     nan                   nan                    0.77                        0.71             nan             nan              0.77              nan               0.62             9.50            8.53          0.78         2.83            0.92          0.80      0.77            0.50         0.09                0.85                   1.12                   1.23        0.18             0.75           0.35   
2      0.03             0.04       7.51      3.12      3.37     0.80          3.68               1.31    0.77       0.29                 0.51          4.89       0.07           6.47       0.84        0.67    1.25              0.42         0.47               2.17               0.66                  2.19     0.41                1.25           0.54                     0.66               0.61         0.21             0.31       0.52                   0.97         0.41             0.46         0.29               0.91              0.69             nan                   nan                    1.74                   nan                    0.44                        0.21             nan             nan              0.25              nan               0.27             0.50            6.68          0.59         0.48            0.92          0.93      0.39             nan         0.58                0.02                   1.29                    nan        2.21             0.32           0.58   
3  

In [109]:
Path('./results/ratio').mkdir(exist_ok = True)
final_ratio.to_csv("./results/ratio/ratio_with_std_9_16.csv", index=False, header=True)